# 이미지 설명 요청하기

## 인터넷에 있는 이미지 사진 설명 요청하기
- 원하는 이미지 링크 사용
- 단, API 호출을 통해 서버에 접근 가능한 이미지 링크를 사용하기

In [1]:
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("OPEN_API_KEY")  # 환경 변수에서 API 키를 가져오기

client = OpenAI(api_key=api_key)  # 오픈AI 클라이언트의 인스턴스 생성

messages = [
    {
        "role": "user",
        "content": [
            {"type": "text", "text": "이 이미지에 대해 설명해주세요."},
            {
                "type": "image_url",
                "image_url": {
                    "url": "https://recipe1.ezmember.co.kr/cache/recipe/2016/06/05/1ba7e34cf0daf694f09a3a9539ebdb161.jpg",
                },
            },
        ],
    }
]

response = client.chat.completions.create(
    model="gpt-4o",  # 응답 생성에 사용할 모델 지정
    messages=messages # 대화 기록을 입력으로 전달
)

response


ChatCompletion(id='chatcmpl-CQm50RE7u0wOan2EVs6oktDgDHDFV', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='이미지에는 오므라이스가 담겨 있습니다. 부드러운 계란 지단 안에 밥이 들어 있으며, 위에는 케첩이 뿌려져 있습니다. 밥에는 잘게 썬 햄과 채소들이 포함되어 있는 듯 보입니다. 접시의 뒤쪽에는 브로콜리와 얇게 썬 배가 함께 담겨 있습니다. 맛있게 보이는 한 끼 식사입니다.', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1760497382, model='gpt-4o-2024-08-06', object='chat.completion', service_tier='default', system_fingerprint='fp_cbf1785567', usage=CompletionUsage(completion_tokens=96, prompt_tokens=779, total_tokens=875, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

## 내가 가진 이미지로 설명 요청하기

### base64 인코딩
- 바이너리 데이터 HTTP Request 전송을 위해 인코딩 진행

In [3]:
import base64

# Function to encode the image
def encode_image(path):
    with open(path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")
    
image_path = "C:/Users/gunwo/anaconda3/envs/TEST/llm/06장/image/mart.jpg"

# 이미지를 base64로 인코딩
base64_image = encode_image(image_path)

print(base64_image[0:100])

/9j/4AAUSkZJRgABAQEBLAEsAABBTVBG/+EJxEV4aWYAAE1NACoAAAAIAA0BDwACAAAABgAAAKoBEAACAAAACgAAALABEgADAAAA


### GPT API 호출 및 결과 출력

In [4]:
messages = [
    {
        "role": "user",
        "content": [
            {"type": "text", "text": "이 이미지에 대해 설명해주세요."},
            {
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/jpeg;base64,{base64_image}",
                },
            },
        ],
    }
]

response = client.chat.completions.create(
    model="gpt-4o",  # 응답 생성에 사용할 모델 지정
    messages=messages # 대화 기록을 입력으로 전달
)

response.choices[0].message.content

'이 이미지는 식료품 가게의 한 코너를 보여줍니다. 선반에는 다양한 즉석 라면 제품들이 진열되어 있습니다. 빨간색 패키지와 컵라면 용기가 눈에 띄며, 서로 다른 브랜드와 맛이 있는 것으로 보입니다. 아래쪽 선반에는 쌀국수와 같은 면 제품들이 포장되어 있습니다. 바닥은 회색 타일로 덮여 있습니다.'

### 여러 이미지 비교 분석 요청

In [ ]:
red_dumpling_base64 = encode_image("C:/Users/gunwo/anaconda3/envs/TEST/llm/06장/red_dumpling.jpeg")
dumpling_soup_base64 = encode_image("C:/Users/gunwo/anaconda3/envs/TEST/llm/06장/dumpling_soup.jpeg")

messages = [
    {
        "role": "user",
        "content": [
            {"type": "text", "text": "두 음식의 차이점을 설명해주세요."},
            {
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/jpeg;base64,{red_dumpling_base64}",
                },
            },
            {
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/jpeg;base64,{dumpling_soup_base64}",
                },
            },
        ],
    }
]

response = client.chat.completions.create(
    model="gpt-4o",  # 응답 생성에 사용할 모델 지정
    messages=messages # 대화 기록을 입력으로 전달
)

response.choices[0].message.content

'첫 번째 사진은 매콤한 소스에 담긴 완탕 스타일의 만두 요리입니다. 주로 빨간 고추 기름이 들어가 있어 매콤한 맛을 냅니다. 두 번째 사진은 국물에 담겨있는 만두 요리로, 만두국과 비슷해 보입니다. 이 요리에는 계란 지단과 같은 고명이 추가되어 있으며, 상대적으로 맑고 깔끔한 맛을 제공합니다. 두 요리는 조리 방식과 소스의 종류에 따라 맛과 식감이 다릅니다.'

## GPT 비전의 한계 알아보기

In [ ]:
oecd_rnd_2021_base64 = encode_image("C:/Users/gunwo/anaconda3/envs/TEST/llm/06장/oecd_rnd_2021_large.png")
oecd_rnd_2022_base64 = encode_image("C:/Users/gunwo/anaconda3/envs/TEST/llm/06장/oecd_rnd_2022_large.png")

messages = [
    {
        "role": "user",
        "content": [
            {"type": "text", "text": "첫번째는 2021년 데이터이고, 두번째는 2022년 데이터입니다. 이 데이터에 대해 설명해주세요. 어떤 변화가 있었나요? 한국 중심으로 설명해주세요."},
            {
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/jpeg;base64,{oecd_rnd_2021_base64}",
                },
            },
            {
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/jpeg;base64,{oecd_rnd_2022_base64}",
                },
            },
        ],
    }
]

response = client.chat.completions.create(
    model="gpt-4o",  # 응답 생성에 사용할 모델 지정
    messages=messages # 대화 기록을 입력으로 전달
)

response.choices[0].message.content


'이 두 그래프는 2021년과 2022년의 주요 국가별 연구개발비와 GDP 대비 연구개발비 비중을 비교하고 있습니다. 여기서는 한국을 중심으로 변화에 대해 설명하겠습니다.\n\n### 한국의 연구개발비 변화:\n- **2021년**: 연구개발비는 89,282백만 달러였고, GDP 대비 연구개발비 비중은 4.93%였습니다.\n- **2022년**: 연구개발비는 91,013백만 달러로 증가했지만 GDP 대비 비율은 4.93%에서 4.91%로 소폭 감소했습니다.\n\n### 글로벌 비교:\n- **미국**의 연구개발비와 비중이 가장 높으며, 2022년에도 연구개발 투자가 크게 증가했습니다.\n- **중국**과 **일본**도 두드러진 연구개발비를 보유하고 있지만, 특히 중국은 두 해 모두 많은 연구개발 투자를 했습니다.\n- **스위스**, **스웨덴** 등 일부 국가들은 GDP 대비 연구개발비 비중이 높으며, 한국보다도 높은 비율을 보입니다.\n\n### 주요 변화:\n- 글로벌 전반적으로 연구개발비가 증가하는 추세를 보였으며, GDP 대비 비중은 다소 변동이 있었습니다.\n- 각국의 연구개발 우선순위가 다양하게 반영되며, 한국은 꾸준히 높은 비중을 유지하고 있습니다.\n\n이 데이터는 주로 각국의 경제 상황과 정책 방향의 변화를 반영하여 연구개발의 절대적 중요성과 전략적 우선 순위를 보여줍니다.'

In [ ]:
oecd_rnd_2021_base64 = encode_image("C:/Users/gunwo/anaconda3/envs/TEST/llm/06장/oecd_rnd_2021_medium.png")
oecd_rnd_2022_base64 = encode_image("C:/Users/gunwo/anaconda3/envs/TEST/llm/06장/oecd_rnd_2022.png")

messages = [
    {
        "role": "user",
        "content": [
            {"type": "text", "text": "첫번째는 2021년 데이터이고, 두번째는 2022년 데이터입니다. 이 데이터에 대해 설명해주세요. 어떤 변화가 있었나요? 한국 중심으로 설명해주세요."},
            {
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/jpeg;base64,{oecd_rnd_2021_base64}",
                },
            },
            {
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/jpeg;base64,{oecd_rnd_2022_base64}",
                },
            },
        ],
    }
]

response = client.chat.completions.create(
    model="gpt-4o",  # 응답 생성에 사용할 모델 지정
    messages=messages # 대화 기록을 입력으로 전달
)

response.choices[0].message.content

'2021년과 2022년 데이터를 비교하면 한국의 연구개발비와 GDP 대비 연구개발비 비중에 다음과 같은 변화가 있었습니다.\n\n1. **연구개발비**:\n   - 2021년: 약 121,739백만 달러\n   - 2022년: 약 133,867백만 달러\n   - 증가: 약 12,128백만 달러\n   \n2. **GDP 대비 연구개발비 비중**:\n   - 2021년: 4.93%\n   - 2022년: 5.21%\n   - 증가: 0.28%\n\n한국은 연구개발비 총액과 GDP 대비 연구개발비 비중 모두 증가했습니다. 이는 한국이 과학기술 분야에 대한 투자를 증가시키고 있음을 나타냅니다. 이러한 증가는 연구개발 역량 강화와 혁신 추진에 긍정적인 기여를 할 수 있습니다.'